In [ ]:
using CSV
using DataFrames 
using Pipe

In [ ]:
using MLJ
using OutlierDetection
using OutlierDetectionNeighbors: KNNDetector
using OutlierDetectionData: ODDS

In [ ]:
ENV["LINES"], ENV["COLUMNS"] = 15, 200

In [ ]:
ls_loc = "../dataset/dataset/last_year/"

In [ ]:
exc = ["Column1", "土地位置建物門牌", "編號", "非都市土地使用分區", "非都市土地使用編定", 
    "都市土地使用分區", "交易筆棟數", "交易年月日", "建築完成年月", "備註", "移轉編號", "period"]
println("")

In [ ]:
ls_tp = @pipe CSV.File(ls_loc*"LastYear_tp.csv") |> 
    DataFrame |>
    select(_, Not(exc))
println("")

In [ ]:
ls_ks = @pipe CSV.File(ls_loc*"LastYear_ks.csv") |> 
    DataFrame |>
    select(_, Not(exc))
println("")

In [ ]:
ls_nt = @pipe CSV.File(ls_loc*"LastYear_nt.csv")  |> 
    DataFrame |>
    select(_, Not(exc))
println("")

In [ ]:
ls_tc = @pipe CSV.File(ls_loc*"LastYear_tc.csv") |> 
    DataFrame |>
    select(_, Not(exc))
println("")

In [ ]:
ls_tn = @pipe CSV.File(ls_loc*"LastYear_tn.csv")  |> 
    DataFrame |>
    select(_, Not(exc))
println("")

In [ ]:
ls_ty = @pipe CSV.File(ls_loc*"LastYear_ty.csv") |> 
    DataFrame |>
    select(_, Not(exc))
println("")

In [377]:
function prob(x, df)
    nrow(x) / sum(nrow(df))
end

prob (generic function with 2 methods)

In [473]:
function cate_table(df)
    # find cate var
    int =  [eltype.(eachcol(df)) .== Int] + 
        [eltype.(eachcol(df)) .== Union{Missing, Int}]
    flo = [eltype.(eachcol(df)) .== Float64] 
    num_var = Bool.((int + flo)[1])
    cat_var = .!num_var
    cat_df = df[:, cat_var]
    for i in 1:length(names(cat_df))
        var = names(cat_df)[i]
        tdf = @pipe groupby(cat_df, var) |>
            combine(_, nrow)
        total = sum(tdf[:, 2])
        @pipe tdf |>
            transform!(_, :nrow => x -> x / total * 100) |>
            rename!(_, [:nrow, :nrow_function] .=> [:n, :prob])
        @show tdf
    end
end

cate_table (generic function with 2 methods)

In [474]:
cate_table(ls_tp)

total = 20294
tdf = 12×3 DataFrame
 Row │ 鄉鎮市區  n      prob
     │ String15  Int64  Float64
─────┼───────────────────────────
   1 │ 中山區     2975  14.6595
   2 │ 北投區     2056  10.1311
   3 │ 士林區     1868   9.20469
   4 │ 大安區     1869   9.20962
   5 │ 內湖區     2485  12.245
   6 │ 中正區     1159   5.71105
   7 │ 文山區     1627   8.01715
   8 │ 信義區     1673   8.24382
   9 │ 松山區     1274   6.27772
  10 │ 南港區     1016   5.00641
  11 │ 大同區      958   4.72061
  12 │ 萬華區     1334   6.57337
total = 20294
tdf = 5×3 DataFrame
 Row │ 交易標的              n      prob
     │ String31              Int64  Float64
─────┼────────────────────────────────────────
   1 │ 車位                   1033   5.09017
   2 │ 土地                   1721   8.48034
   3 │ 房地(土地+建物)       11238  55.376
   4 │ 建物                    103   0.507539
   5 │ 房地(土地+建物)+車位   6199  30.546
total = 20294
tdf = 215×3 DataFrame
 Row │ 移轉層次                           n      prob
     │ String?                            Int64  Float64
─────┼─────

total = 20294
tdf = 36×3 DataFrame
 Row │ 總樓層數   n      prob
     │ String15?  Int64  Float64
─────┼──────────────────────────────
   1 │ 十七層       161   0.793338
   2 │ missing     1922   9.47078
   3 │ 十二層      1972   9.71716
   4 │ 六層         486   2.3948
   5 │ 四層        2107  10.3824
   6 │ 九層         566   2.789
   7 │ 十八層       352   1.7345
   8 │ 十三層       943   4.64669
   9 │ 十層         686   3.38031
  10 │ 一層          54   0.266088
  11 │ 七層        1987   9.79107
  12 │ 八層         556   2.73973
  13 │ 五層        2919  14.3836
  14 │ 三十一層      12   0.0591308
  15 │ 三層         281   1.38465
  16 │ 十一層       904   4.45452
  17 │ 十九層       253   1.24667
  18 │ 十四層      1401   6.90352
  19 │ 二十五層      32   0.157682
  20 │ 十六層       222   1.09392
  21 │ 二層         246   1.21218
  22 │ 十五層      1171   5.77018
  23 │ 二十四層     262   1.29102
  24 │ 二十九層      12   0.0591308
  25 │ 二十一層     241   1.18754
  26 │ 二十層        97   0.477974
  27 │ 二十三層     138   0.680004
  28 │ 四十二層      12   

In [ ]:
using DecisionTree
features, labels = load_data("iris")  

In [ ]:
# train full-tree classifier
model = build_tree(labels, features)
# prune tree: merge leaves having >= 90% combined purity (default: 100%)
model = prune_tree(model, 0.9)
# pretty print of the tree, to a depth of 5 nodes (optional)
print_tree(model, 5)

In [ ]:
preds = apply_tree(model, features)
DecisionTree.confusion_matrix(labels, preds)
apply_tree_proba(model, [5.9,3.0,5.1,1.9], ["Iris-setosa", "Iris-versicolor", "Iris-virginica"])

In [ ]:
n_folds=3
accuracy = nfoldCV_tree(labels, features, n_folds)